In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

nltk.download('punkt')           
nltk.download('averaged_perceptron_tagger') 
nltk.download('stopwords')
nltk.download('wordnet')

#https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

c:\Users\rishu\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\rishu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rishu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rishu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_da

True

In [ ]:

np.random.seed(123)
train = pd.read_csv('Data/train/train.tsv',sep='\t')
test = pd.read_csv('Data/test/test.tsv', sep='\t')
X_train = train.iloc[:, 0:-1]
y_train = train.iloc[:, -1]
x_test = test


X_train['Phrase'].dropna(inplace=True) #remove blank rows
X_train['Phrase'] = [word.lower() for word in X_train['Phrase']] #make everything lowercase
X_train['Phrase'] = [word_tokenize(pharse) for pharse in X_train['Phrase']] #Each phrase is broken into a set of words


#
#All FOR TRAINING DATA
#

#Trying to see if each word is a noun, verb or adjective
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(X_train['Phrase']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    X_train.loc[index,'text_final'] = str(Final_words)
 
# 
#SAME THING FOR TEST DATA
#






,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"[a, series, of, escapades, demonstrating, the,...",1
1,2,1,"[a, series, of, escapades, demonstrating, the,...",2
2,3,1,"[a, series]",2
3,4,1,[a],2
4,5,1,[series],2
...,...,...,...,...
156055,156056,8544,"[hearst, 's]",2
156056,156057,8544,"[forced, avuncular, chortles]",1
156057,156058,8544,"[avuncular, chortles]",3
156058,156059,8544,[avuncular],2


In [4]:
y_train = y_train.to_numpy() #Turn the df into an array

X_train



NameError: name 'y_train' is not defined

In [3]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
#Test_X_Tfidf = Tfidf_vect.transform(Test_X)

NameError: name 'X_train' is not defined